In [3]:
import torch
import glob
from torcheval.metrics.functional import multiclass_accuracy
from numpy import genfromtxt
from torch.nn import RNN, LSTM, GRU
from torch.utils.data import DataLoader


from model import *
from dataset import MyDataset
from utils import read_data

In [4]:
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

In [5]:
test_path = "../nn4_data/UCI HAR Dataset/test/Inertial Signals/"
test_x_data = read_data(test_path).to(device=device)
test_y_path = "../nn4_data/UCI HAR Dataset/test/y_test.txt"
test_y_data = (torch.tensor(genfromtxt(test_y_path)) - 1).to(device=device)


test_dataset = MyDataset(test_x_data, test_y_data)
test_dataloader = DataLoader(test_dataset, batch_size=len(test_dataset))

In [57]:
input_size = 9
num_layers = 1
hidden_size = 6

indexed_action = {0: "Walking", 1: "Walking upstairs", 2: "Walking downstairs", 3: "Sitting", 4: "Standing", 5: "Laying"}

model = GRURNN(input_size, num_layers, hidden_size)
model.load_state_dict(torch.load(f"GRU RNN.pt"))
model = model.to(device=device)
counter = 0
pairs = {}
for i in range(len(test_dataset)):
    hidden = model.init_hidden().to(device=device)
    predicted = model(test_dataset[i][0].unsqueeze(0), hidden)[0].squeeze().argmax()
    if predicted != test_dataset[i][1]:
        if (indexed_action[int(predicted)], indexed_action[int(test_dataset[i][1])]) in pairs:
            pairs[(indexed_action[int(predicted)], indexed_action[int(test_dataset[i][1])])] += 1
        else:
            pairs[(indexed_action[int(predicted)], indexed_action[int(test_dataset[i][1])])] = 1
    else:
        counter += 1
print(counter/len(test_dataset))

for item, count in pairs.items():
    print(item, count)

0.5945028842891076
('Walking', 'Standing') 8
('Standing', 'Sitting') 154
('Walking downstairs', 'Walking') 162
('Standing', 'Walking') 91
('Walking upstairs', 'Walking') 39
('Standing', 'Walking downstairs') 62
('Walking', 'Walking downstairs') 140
('Walking upstairs', 'Walking downstairs') 26
('Standing', 'Walking upstairs') 95
('Walking', 'Walking upstairs') 139
('Walking downstairs', 'Walking upstairs') 138
('Walking downstairs', 'Standing') 7
('Sitting', 'Standing') 60
('Sitting', 'Walking') 34
('Sitting', 'Walking downstairs') 16
('Walking', 'Sitting') 3
('Sitting', 'Walking upstairs') 20
('Walking upstairs', 'Standing') 1


In [53]:
for item in sorted(pairs, key=lambda x: pairs[x]):
    print(item, pairs[item])

('Walking', 'Sitting') 1
('Walking upstairs', 'Standing') 6
('Walking', 'Standing') 9
('Walking downstairs', 'Walking upstairs') 16
('Walking downstairs', 'Walking') 20
('Walking upstairs', 'Laying') 27
('Sitting', 'Walking upstairs') 33
('Sitting', 'Walking downstairs') 62
('Sitting', 'Walking') 69
('Walking upstairs', 'Walking downstairs') 73
('Sitting', 'Standing') 90
('Walking upstairs', 'Walking') 92
('Standing', 'Walking upstairs') 97
('Standing', 'Walking downstairs') 98
('Standing', 'Sitting') 126
('Standing', 'Walking') 128
('Walking', 'Walking downstairs') 145
('Walking', 'Walking upstairs') 154


In [49]:
for item, count in pairs:
    print(item, count)

Walking Sitting
Walking upstairs Standing
Walking Standing
Walking downstairs Walking upstairs
Walking downstairs Walking
Walking upstairs Laying
Sitting Walking upstairs
Sitting Walking downstairs
Sitting Walking
Walking upstairs Walking downstairs
Sitting Standing
Walking upstairs Walking
Standing Walking upstairs
Standing Walking downstairs
Standing Sitting
Standing Walking
Walking Walking downstairs
Walking Walking upstairs


In [56]:
input_size = 9
num_layers = 1
hidden_size = 6


rnn_models = {"classic RNN": SimpleRNN(input_size, num_layers, hidden_size),
              "bidirectional RNN": BidirectionalRNN(input_size, num_layers, hidden_size),
              "LSTM RNN": LSTMRNN(input_size, num_layers, hidden_size),
              "GRU RNN": GRURNN(input_size, num_layers, hidden_size),
              }

for name, model in rnn_models.items():
    print(name)
    model.load_state_dict(torch.load(f"{name}.pt"))
    model = model.to(device=device)
    for item, label in test_dataloader:
        hidden = model.init_hidden().to(device=device)
        predicted = model(item, hidden)[0].argmax(1)
        print(multiclass_accuracy(predicted, label))

classic RNN
tensor(0.5022, device='cuda:0')
bidirectional RNN
tensor(0.5131, device='cuda:0')
LSTM RNN
tensor(0.5565, device='cuda:0')
GRU RNN
tensor(0.5707, device='cuda:0')


In [ ]:
# nlayers = 4
#tensor(0.6373, device='cuda:0')
#tensor(0.7723, device='cuda:0')
#tensor(0.4605, device='cuda:0')
#tensor(0.5972, device='cuda:0')